In [2]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



In [3]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [5]:

batch_size = 128
epochs = 1

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


422/422 [==============================] - 88s 207ms/step - loss: 0.7344 - accuracy: 0.7734 - val_loss: 0.0822 - val_accuracy: 0.9768


In [6]:
test = model.predict(x_test)

In [8]:
tf.saved_model.save(model, 'models/1/')

INFO:tensorflow:Assets written to: models/1/assets


In [24]:
import requests
import scaleout.auth as sa

In [31]:
inp = {"inputs": x_test[0:10].tolist()}

In [43]:
token, config = sa.get_token()
res = requests.post('https://tensorflow-serving-test-eju-9ce4.studio-dev.local.stackn.dev/v1/models/models:predict', json=inp, headers={"Authorization": "Bearer "+token})

In [44]:
res.json()

{'outputs': [[1.64117466e-07,
   1.71092374e-09,
   9.02497686e-06,
   8.33915692e-06,
   2.56994287e-10,
   4.90337682e-09,
   1.96727205e-12,
   0.999978185,
   6.85942538e-08,
   4.2300062e-06],
  [0.000341361563,
   0.00047096191,
   0.997864068,
   4.03026643e-05,
   2.21642527e-09,
   5.41947884e-07,
   0.00105721201,
   4.4137749e-10,
   0.000225375145,
   1.8258178e-11],
  [4.85109e-05,
   0.995219767,
   0.000300258427,
   6.68863941e-05,
   0.00218053651,
   5.24084635e-06,
   0.000829518249,
   0.000645498571,
   0.000651024,
   5.27858465e-05],
  [0.999343097,
   3.01201508e-10,
   2.77711642e-05,
   1.25704389e-07,
   8.76815136e-08,
   8.30702447e-06,
   0.000598079234,
   7.00484225e-06,
   7.81792551e-06,
   7.66947687e-06],
  [3.94258359e-06,
   4.71371351e-07,
   7.7931345e-06,
   8.34848265e-07,
   0.997844815,
   3.93695672e-07,
   3.69919435e-05,
   2.47607659e-05,
   1.13509641e-05,
   0.00206880225],
  [5.85317175e-05,
   0.994883716,
   0.000249574747,
   9.5520